<a href="https://colab.research.google.com/github/suparuek2405/Stock-screening-with-python/blob/main/Group_assignment_Tacocat_set_yfinance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#install libraries
# !pip install pandas
# !pip install numpy
# !pip install yfinance

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from google.colab import drive

In [4]:
# version 1
def rename_columns_to_year_end(df):
    df_rt = df.copy()
    years = df_rt.columns.year
    year_end_dates = [pd.Timestamp(f"{year}-12-31") for year in years]
    df_rt.columns = year_end_dates
    return df_rt

def get_balance_sheet(ticker):
  balance_sheet_df = pd.DataFrame()
  n = len(ticker)
  print("start getting balance sheet")
  for c,i in enumerate(ticker):
    #retrive data from yfinance
    stock = yf.Ticker(i)
    raw = stock.balancesheet

    #rename column
    bs = rename_columns_to_year_end(raw)

      #reset index and rename it
    bs.reset_index(inplace=True)
    bs.rename(columns={'index': 'data'}, inplace=True)
    bs['symbol'] = i.split('.')[0]

    # Check for duplicates in the 'data' column and drop them
    bs = bs[~bs.duplicated(subset=['data'])]
    bs.set_index(['symbol', 'data'], inplace=True)

    #change type data of column for better look
    bs.columns = pd.to_datetime(bs.columns)

    #append to dataframe
    balance_sheet_df = pd.concat([balance_sheet_df, bs])
    print(f"{c+1}/{n} {i}")
  print("finish getting balance sheet")
  return balance_sheet_df

def get_income_statement(ticker):
  income_statement_df = pd.DataFrame()
  n = len(ticker)
  print("start getting income statement")
  for c,i in enumerate(ticker):
    #retrive data from yfinance
    stock = yf.Ticker(i)
    raw = stock.financials

    #rename column
    is_df = rename_columns_to_year_end(raw)

    #reset index and rename it
    is_df.reset_index(inplace=True)
    is_df.rename(columns={'index': 'data'}, inplace=True)
    is_df['symbol'] = i.split('.')[0]

    # Check for duplicates in the 'data' column and drop them
    is_df = is_df[~is_df.duplicated(subset=['data'])]
    is_df.set_index(['symbol', 'data'], inplace=True)

    #change type data of column for better look
    is_df.columns = pd.to_datetime(is_df.columns)

    #append to dataframe
    income_statement_df = pd.concat([income_statement_df, is_df])
    print(f"{c+1}/{n} {i}")
  print("finish getting income statement")
  return income_statement_df

def get_cashflow_statement(ticker):
  cashflow_statement_df = pd.DataFrame()
  n = len(ticker)
  print("start getting cashflow statement")
  for c,i in enumerate(ticker):
    #retrive data from yfinance
    stock = yf.Ticker(i)
    raw = stock.cashflow

    #rename column
    cf_df = rename_columns_to_year_end(raw)

    #reset index and rename it
    cf_df.reset_index(inplace=True)
    cf_df.rename(columns={'index': 'data'}, inplace=True)
    cf_df['symbol'] = i.split('.')[0]

    # Check for duplicates in the 'data' column and drop them
    cf_df = cf_df[~cf_df.duplicated(subset=['data'])]
    cf_df.set_index(['symbol', 'data'], inplace=True)

    #change type data of column for better look
    cf_df.columns = pd.to_datetime(cf_df.columns)

    #append to dataframe
    cashflow_statement_df = pd.concat([cashflow_statement_df, cf_df])
    print(f"{c+1}/{n} {i}")
  print("finish getting cashflow statement")
  return cashflow_statement_df

def get_monthly_stock_prices(ticker,start_date = "2022-01-01", end_date="2024-09-30"):
  stock = yf.Ticker(ticker)
  data = stock.history(start=start_date, end=end_date, interval="1mo")
  data = data.set_index(pd.to_datetime(data.index))

  # Resample to the start of each month
  monthly_prices = data['Close'].resample('MS').first()
  monthly_prices = pd.DataFrame(monthly_prices)
  monthly_prices['symbol'] = ticker.split('.')[0]
  monthly_prices.index = monthly_prices.index.to_period('M').to_timestamp(how='start')
  return monthly_prices

def get_dividend_data(ticker, start_date="2022-01-01", end_date="2024-09-30"):
  stock = yf.Ticker(ticker)
  dividends = stock.dividends
  # Filter dividends based on the specified date range
  dividends = dividends[(dividends.index >= start_date) & (dividends.index <= end_date)]
  if not dividends.empty:
    dividends_df = pd.DataFrame(dividends)
    dividends_df['symbol'] = ticker.split('.')[0]
    dividends_df = dividends_df.rename(columns={'Dividends':'amount'})
    dividends_df.index = dividends_df.index.to_period('M').to_timestamp(how='start')
    return dividends_df

def get_financial_data(ticker):
  balance_sheet = get_balance_sheet(ticker)
  income_statement = get_income_statement(ticker)
  cashflow_statement = get_cashflow_statement(ticker)
  return balance_sheet, income_statement, cashflow_statement

In [8]:
#df = pd.read_csv('/content/drive/My Drive/Tools/Group Assignment/set_scraped.csv')
df = pd.read_csv('https://raw.githubusercontent.com/suparuek2405/Stock-screening-with-python/main/set_scraped.csv')

In [10]:
bkk = df.copy()
bkk = bkk[['symbol', 'text-start', 'text-start 2', 'symbol 2', 'symbol 3']]
bkk.columns = ['symbol', 'company_name', 'market', 'industry', 'sector']
bkk['ticker'] = bkk['symbol'] + '.BK'
bkk.head(10)

,symbol,company_name,market,industry,sector,ticker
0,24CS,บริษัท ทเวนตี้ โฟร์ คอน แอนด์ ซัพพลาย จำกัด (ม...,mai,PROPCON,NaN,24CS.BK
1,2S,บริษัท 2 เอส เมทัล จำกัด (มหาชน),SET,INDUS,STEEL,2S.BK
2,3K-BAT,บริษัท ไทย เอ็นเนอร์จี สโตเรจ เทคโนโลยี จำกัด ...,SET,INDUS,AUTO,3K-BAT.BK
3,A,บริษัท อารียา พรอพเพอร์ตี้ จำกัด (มหาชน),SET,PROPCON,PROP,A.BK
4,A5,บริษัท แอสเซท ไฟว์ กรุ๊ป จำกัด (มหาชน),mai,PROPCON,NaN,A5.BK
5,AAI,บริษัท เอเชี่ยน อะไลอันซ์ อินเตอร์เนชั่นแนล จำ...,SET,AGRO,FOOD,AAI.BK
6,AAV,บริษัท เอเชีย เอวิเอชั่น จำกัด (มหาชน),SET,SERVICE,TRANS,AAV.BK
7,ABM,บริษัท เอเชีย ไบโอแมส จำกัด (มหาชน),mai,RESOURC,NaN,ABM.BK
8,ACAP,บริษัท เอเชีย แคปปิตอล กรุ๊ป จำกัด (มหาชน),mai,FINCIAL,NaN,ACAP.BK
9,ACC,บริษัท แอดวานซ์ คอนเนคชั่น คอร์ปอเรชั่น จำกัด ...,SET,RESOURC,ENERG,ACC.BK


In [ ]:
set_list = bkk['ticker']
ticker_error = ['DIMET.BK'] #exclude error ticker
set_list = set_list[set_list.isin(ticker_error) == False]

In [ ]:
#run this code to start retrive data from yfinance
bl_st, ic_st, cf_st = get_financial_data(set_list)

start getting balance sheet
1/911 24CS.BK
2/911 2S.BK
3/911 3K-BAT.BK
4/911 A.BK
5/911 A5.BK
6/911 AAI.BK
7/911 AAV.BK
8/911 ABM.BK
9/911 ACAP.BK
10/911 ACC.BK
11/911 ACE.BK
12/911 ACG.BK
13/911 ADB.BK
14/911 ADD.BK
15/911 ADVANC.BK
16/911 ADVICE.BK
17/911 AE.BK
18/911 AEONTS.BK
19/911 AF.BK
20/911 AFC.BK
21/911 AGE.BK
22/911 AH.BK
23/911 AHC.BK
24/911 AI.BK
25/911 AIE.BK
26/911 AIMCG.BK
27/911 AIMIRT.BK
28/911 AIRA.BK
29/911 AIT.BK
30/911 AJ.BK
31/911 AJA.BK
32/911 AKP.BK
33/911 AKR.BK
34/911 AKS.BK
35/911 ALLA.BK
36/911 ALLY.BK
37/911 ALPHAX.BK
38/911 ALT.BK
39/911 ALUCON.BK
40/911 AMA.BK
41/911 AMANAH.BK
42/911 AMARC.BK
43/911 AMARIN.BK
44/911 AMATA.BK
45/911 AMATAR.BK
46/911 AMATAV.BK
47/911 AMC.BK
48/911 AMR.BK
49/911 ANAN.BK
50/911 ANI.BK
51/911 AOT.BK
52/911 AP.BK
53/911 APCO.BK
54/911 APCS.BK
55/911 APEX.BK
56/911 APO.BK
57/911 APP.BK
58/911 APURE.BK
59/911 AQUA.BK
60/911 ARIN.BK
61/911 ARIP.BK
62/911 ARROW.BK
63/911 AS.BK
64/911 ASAP.BK
65/911 ASEFA.BK
66/911 ASIA.BK
67/911 AS

In [ ]:
# run this code below to download file
# bl_st.to_csv('balance_sheet.csv')
# ic_st.to_csv('income_statement.csv')
# cf_st.to_csv('cashflow_statement.csv')

# from google.colab import files
# files.download('balance_sheet.csv')
# files.download('income_statement.csv')
# files.download('cashflow_statement.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

------

## After get data


In [13]:
bl = pd.read_csv('https://raw.githubusercontent.com/suparuek2405/Stock-screening-with-python/refs/heads/main/balance_sheet.csv')
ic = pd.read_csv('https://raw.githubusercontent.com/suparuek2405/Stock-screening-with-python/refs/heads/main/income_statement.csv')
cf = pd.read_csv('https://raw.githubusercontent.com/suparuek2405/Stock-screening-with-python/refs/heads/main/cashflow_statement.csv')

In [17]:
bkk

,symbol,company_name,market,industry,sector,ticker
0,24CS,บริษัท ทเวนตี้ โฟร์ คอน แอนด์ ซัพพลาย จำกัด (ม...,mai,PROPCON,NaN,24CS.BK
1,2S,บริษัท 2 เอส เมทัล จำกัด (มหาชน),SET,INDUS,STEEL,2S.BK
2,3K-BAT,บริษัท ไทย เอ็นเนอร์จี สโตเรจ เทคโนโลยี จำกัด ...,SET,INDUS,AUTO,3K-BAT.BK
3,A,บริษัท อารียา พรอพเพอร์ตี้ จำกัด (มหาชน),SET,PROPCON,PROP,A.BK
4,A5,บริษัท แอสเซท ไฟว์ กรุ๊ป จำกัด (มหาชน),mai,PROPCON,NaN,A5.BK
...,...,...,...,...,...,...
907,YONG,บริษัท ยงคอนกรีต จำกัด (มหาชน),mai,PROPCON,NaN,YONG.BK
908,YUASA,บริษัท ยัวซ่าแบตเตอรี่ ประเทศไทย จำกัด (มหาชน),mai,INDUS,NaN,YUASA.BK
909,ZAA,บริษัท ซาเล็คต้า จำกัด (มหาชน),SET,SERVICE,MEDIA,ZAA.BK
910,ZEN,บริษัท เซ็น คอร์ปอเรชั่น กรุ๊ป จำกัด (มหาชน),SET,AGRO,FOOD,ZEN.BK


In [14]:
bl.data.unique()

array(['Treasury Shares Number', 'Ordinary Shares Number', 'Share Issued',
       'Net Debt', 'Total Debt', 'Tangible Book Value',
       'Invested Capital', 'Working Capital', 'Net Tangible Assets',
       'Capital Lease Obligations', 'Common Stock Equity',
       'Total Capitalization', 'Total Equity Gross Minority Interest',
       'Stockholders Equity', 'Other Equity Interest',
       'Gains Losses Not Affecting Retained Earnings',
       'Other Equity Adjustments', 'Retained Earnings',
       'Additional Paid In Capital', 'Capital Stock', 'Common Stock',
       'Total Liabilities Net Minority Interest',
       'Total Non Current Liabilities Net Minority Interest',
       'Other Non Current Liabilities', 'Employee Benefits',
       'Non Current Pension And Other Postretirement Benefit Plans',
       'Long Term Debt And Capital Lease Obligation',
       'Long Term Capital Lease Obligation', 'Long Term Debt',
       'Current Liabilities', 'Other Current Liabilities',
       'Current 

In [15]:
ic.data.unique()

array(['Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Normalized EBITDA', 'Total Unusual Items',
       'Total Unusual Items Excluding Goodwill',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBITDA',
       'EBIT', 'Net Interest Income', 'Interest Expense',
       'Interest Income', 'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Diluted Average Shares', 'Basic Average Shares',
       'Diluted EPS', 'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Net Income',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations', 'Tax Provision',
       'Pretax Income', 'Other Income Expense',
       'Other Non Operating Income Expenses', 'Special Income Charges',
       'Gain On Sale Of Ppe', 'Other Special Charges',
       'Gain On Sale Of Security'

In [16]:
cf.data.unique()

array(['Free Cash Flow', 'Repayment Of Debt', 'Issuance Of Debt',
       'Issuance Of Capital Stock', 'Capital Expenditure',
       'End Cash Position', 'Other Cash Adjustment Outside Changein Cash',
       'Beginning Cash Position', 'Changes In Cash',
       'Financing Cash Flow',
       'Cash Flow From Continuing Financing Activities',
       'Net Other Financing Charges', 'Cash Dividends Paid',
       'Common Stock Dividend Paid', 'Net Common Stock Issuance',
       'Common Stock Issuance', 'Net Issuance Payments Of Debt',
       'Net Short Term Debt Issuance', 'Short Term Debt Payments',
       'Short Term Debt Issuance', 'Net Long Term Debt Issuance',
       'Long Term Debt Payments', 'Long Term Debt Issuance',
       'Investing Cash Flow',
       'Cash Flow From Continuing Investing Activities',
       'Net Other Investing Changes', 'Interest Received Cfi',
       'Net Business Purchase And Sale', 'Purchase Of Business',
       'Net Intangibles Purchase And Sale', 'Purchase Of In

### Function for Growth investment scanning

2. Growth Investing Using Return on Equity (ROE) and Earnings Growth
* Return on Equity (ROE): Focus on companies with high and increasing ROE, ideally above 15%. (เช็คความต่อเนื่องของ ROE ด้วยว่าคงที่หรือไม่ บางธุรกิจที่เป็น commodity จะมีค่า ROE สูงเว่อร์บ้างช่วงได้)
* Revenue Growth (Income Statement): Look for consistent revenue growth (at least 10% year-over-year) to ensure the company is expanding its market presence.
* Net Profit Margin (Income Statement): A high and improving profit margin (above 10%) shows the company is efficient at turning revenue into profit.
* Return on Assets (ROA) (Income Statement & Balance Sheet): ROA measures profitability relative to total assets. A higher ROA (above 5%) indicates efficient use of assets to generate earnings.
* Operating Cash Flow Growth (Cash Flow Statement): Look for companies with consistent operating cash flow growth. Growing cash flow ensures that earnings growth is supported by actual cash generation.
* Debt-to-Equity (D/E) Ratio (Balance Sheet): For growth companies, a manageable D/E ratio is important. A ratio under 1 is preferable, showing the company is not overly dependent on debt.
* Objective: Focus on companies that not only have strong profitability metrics like ROE but also show consistent revenue, earnings, and cash flow growth to support long-term expansion. (อาจดูเรื่องของ CAPEX เทียบ ยอดขายเพื่อดูสัดส่วนการลงทุนเพื่อการเติบโตของบริษัท)>> Rank by ROE

In [76]:
#Function for Growth investment scanning
from datetime import datetime

def get_lastdateofyear(start_year, end_year):
    """
    Generate the last date of each year between the start year and end year, inclusive.

    Args:
        start_year (int): The starting year.
        end_year (int): The ending year.

    Returns:
        list: A list of strings, where each string represents the last date of the year (in 'YYYY-MM-DD' format)
              for every year from start_year to end_year, inclusive.
    """
    last_dates = []

    for year in range(start_year, end_year + 1):
        last_date = datetime(year, 12, 31).strftime('%Y-%m-%d')
        last_dates.append(last_date)

    return last_dates

def get_roe(startyear=2020, endyear=2022):
  """
    Calculate the Return on Equity (ROE) for each company symbol for the given years.

    The function retrieves 'Net Income' and 'Stockholders' Equity' from the datasets (`ic` and `bl` respectively),
    merges the data on the 'symbol' column, and calculates ROE for each year as:

    ROE = Net Income / Stockholders' Equity

    Args:
        startyear (int, optional): The starting year for which to calculate ROE. Default is 2020.
        endyear (int, optional): The ending year for which to calculate ROE. Default is 2022.

    Returns:
        pandas.DataFrame: A DataFrame containing the company 'symbol', 'ROE' ratio for each year between startyear
                          and endyear, and an additional 'ratio' column that is set to "ROE".
                          The column structure will be ['symbol', 'ratio', year1, year2, ..., yearN].

    Example:
        >>> df = get_roe(2020, 2022)
        >>> print(df)
          symbol ratio  2020-12-31  2021-12-31  2022-12-31
        0   AAPL   ROE        0.15        0.18        0.20
        1   MSFT   ROE        0.12        0.13        0.14
    """
  #get lastdate of year to retrive
  col = get_lastdateofyear(startyear, endyear)

  #create copy of data to calculate
  net_income = ic[ic.data == 'Net Income'][['symbol', 'data'] + col].copy()
  stockholder_equity = bl[bl.data == 'Stockholders Equity'][['symbol', 'data'] + col].copy()

  #merge table
  df_merge = pd.merge(net_income, stockholder_equity, on='symbol', suffixes=('_ni', '_se'))

  # Calculate ROE: Net Income / Average Stockholders' Equity
  roe_dict = {}
  for year in col:
      # Calculate ROE: Net Income / Stockholders' Equity
      df_merge[f'{year}'] = df_merge[f'{year}_ni'] / df_merge[f'{year}_se']

      # Store the ROE results for the year in the dictionary
      roe_dict[f'{year}'] = df_merge[[f'{year}']]

  #prep return dataframe
  df = df_merge[['symbol'] + [f'{year}' for year in col]].copy()
  df['ratio'] = "ROE"
  return df[['symbol', 'ratio'] + col]

In [72]:
get_roe(2022, 2023)

,symbol,ratio,2022-12-31,2023-12-31
0,24CS,ROE,0.040216,-0.079913
1,2S,ROE,-0.060549,0.079456
2,3K-BAT,ROE,NaN,0.052944
3,A,ROE,0.041458,-0.129684
4,A5,ROE,0.109213,0.361270
...,...,...,...,...
886,YONG,ROE,0.120806,0.139686
887,YUASA,ROE,0.107010,0.111582
888,ZAA,ROE,0.054440,-0.121616
889,ZEN,ROE,0.120169,0.113101


In [66]:
bl

,symbol,data,2019-12-31,2020-12-31,2021-12-31,2022-12-31,2023-12-31,2024-12-31
0,24CS,Treasury Shares Number,NaN,NaN,NaN,NaN,0.0,NaN
1,24CS,Ordinary Shares Number,NaN,430000000.0,430000000.0,430000000.0,430000000.0,NaN
2,24CS,Share Issued,NaN,430000000.0,430000000.0,430000000.0,430000000.0,NaN
3,24CS,Net Debt,NaN,9260000.0,42511083.0,NaN,NaN,NaN
4,24CS,Total Debt,NaN,24520000.0,106415812.0,37403012.0,41120997.0,NaN
...,...,...,...,...,...,...,...,...
76527,ZIGA,Gross Accounts Receivable,NaN,NaN,140935223.0,151137224.0,NaN,NaN
76528,ZIGA,Cash Cash Equivalents And Short Term Investments,NaN,197145469.0,198968251.0,106656345.0,184368020.0,NaN
76529,ZIGA,Cash And Cash Equivalents,NaN,197145469.0,198968251.0,106656345.0,184368020.0,NaN
76530,ZIGA,Cash Equivalents,NaN,196905665.0,197489583.0,106123872.0,184305453.0,NaN


In [37]:
net_income


,symbol,data,2020-12-31,2021-12-31,2022-12-31
22,24CS,Net Income,6940000.0,19455578.0,24494231.0
69,2S,Net Income,410165000.0,737515000.0,-122177000.0
118,3K-BAT,Net Income,-229001875.0,153497275.0,NaN
170,A,Net Income,194865060.0,-345562551.0,134272337.0
223,A5,Net Income,57239210.0,130692766.0,97787455.0
...,...,...,...,...,...
42037,YONG,Net Income,44344600.0,68940193.0,100980244.0
42080,YUASA,Net Income,158177312.0,97682144.0,109415907.0
42128,ZAA,Net Income,-64380958.0,-20798539.0,24970637.0
42177,ZEN,Net Income,-63655000.0,-91573000.0,153907000.0


In [40]:
stockholder_equity

,symbol,data,2020-12-31,2021-12-31,2022-12-31
13,24CS,Stockholders Equity,3.898000e+07,1.369823e+08,6.090699e+08
97,2S,Stockholders Equity,1.757483e+09,2.279259e+09,2.017833e+09
184,3K-BAT,Stockholders Equity,NaN,2.210940e+09,2.251672e+09
272,A,Stockholders Equity,3.573913e+09,3.108638e+09,3.238752e+09
368,A5,Stockholders Equity,6.780680e+08,8.096344e+08,8.953827e+08
...,...,...,...,...,...
76127,YONG,Stockholders Equity,3.646126e+08,4.335528e+08,8.358871e+08
76204,YUASA,Stockholders Equity,8.862245e+08,9.403630e+08,1.022484e+09
76284,ZAA,Stockholders Equity,4.535901e+08,4.344903e+08,4.586840e+08
76364,ZEN,Stockholders Equity,1.211125e+09,1.122448e+09,1.280758e+09
